# Bag Of Words Topic Modeling and Recommender for Ted Talks - JBCS
 - Summer K. Rankin 

once the text has been cleaned, we move onto the next steps
        - 1. Vectorize 
        - 2. Topic modeling
        - 3. Recommender

# 0.0 load the packages

In [0]:
!pip install pyldavis

In [0]:
import nltk, re, pickle, os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation,  TruncatedSVD, NMF
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing  import  StandardScaler


import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt


import pyLDAvis, pyLDAvis.sklearn
from IPython.display import display

# 0.1 Mount Gdrive

In [0]:
from google.colab import drive
drive.mount('/gdrive/')

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).


# 0.2 import the cleaned text

In [0]:
with open('/gdrive/My Drive/JBCS_NLP_intro/cleaned_talks.pkl', 'rb') as picklefile:
    cleaned_talks = pickle.load(picklefile)    

# 1 Vectorize
Vectorization is the important step of turning our words into numbers. There are 2 common methods: count vectorizer, tf-idf. This function takes each word in each document and counts the number of times the word appears. You end up with each word (and n-gram) as your columns and each row is a document, so the data is the frequency of each word in each document. As you can imagine, there will be a large number of zeros in this matrix; we call this a sparse matrix. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

tf-IDF

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

----

For this tutorial the tokenization and vectorization gets bundled together because we are using skleanrn's feature extraction functions. This means we will set the parameters of these functions to tokenize the way we want, include n-grams, and set thresholds for max or min document frequency of a term. https://towardsdatascience.com/hacking-scikit-learns-vectorizers-9ef26a7170af

- takes us from words to numbers 
- create the document-term matrix which is the basis for all modeling
    - row = document, column = word or n-gram, data = word's weight for that document
- we will vectorize in 2 ways 
    1. counting the frequency of each term in each document (**CountVectorizer**)
    2. counting the frequency of each term in each document and weighting by the number of times the term appears in the corpus. Term Frequency * Inverse Document Frequency (**TfidfVectorizer**)


# 1.1 Count Vectorize 
+ Using Sklearn algorithms with text data
+ CountVectorizer: Convert a collection of text documents to a matrix of token counts 
+ This implementation produces a sparse representation
+  **CountVectorizer** is a class; so **vectorizer** below represents an instance of that object
+ note that we can also **lowercase** , remove **stopwords** and search for a certain pattern **token_pattern** (i.e. letters only) by using the parameters of this function. 
+ a specific vocabulary can also be passed to this function. 
+ for more info about the many parameters, see the sklearn docs. 

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

CountVectorizer is a class; so `vectorizer` below represents an instance of that object.


In [0]:
c_vectorizer = CountVectorizer(ngram_range=(1,3), 
                             stop_words='english', 
                             max_df = 0.6, 
                             max_features=2000)

# call `fit` to build the vocabulary and calculate the weights
c_vectorizer.fit(cleaned_talks)

# finally, call `transform` to apply the weights and convert text to a bag of words
count_vect_data = c_vectorizer.transform(cleaned_talks)

# to view the document-term matrix, we can transpose back to a dense array
pd.DataFrame(data = count_vect_data.toarray(), columns=sorted(c_vectorizer.get_feature_names()))

,000,000 dollar,000 people,000 year,10,10 000,10 year,100,100 000,11,12,13,14,15,15 year,150,16,17,18,20,20 percent,20 year,200,20th,20th century,21st,21st century,24,25,30,30 year,300,35,3d,40,40 year,400,45,50,50 percent,...,wire,wisdom,wish,woman,wonder,wonderful,wondering,wood,word,worked,worker,working,world war,worldwide,worried,worry,worse,worst,worth,wow,write,writer,writing,written,wrong,wrote,year ago,year later,year old,year year,yellow,yes,yesterday,york,york city,young,young people,youtube,zero,zone
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,2,0,...,0,0,0,5,0,5,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,7,1,1,0,1,0,0,0,2,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,2,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0
2,1,0,0,0,1,0,0,0,0,3,2,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,0,12,0,0,1,0,0,0,0,1,0,1,0,6,0,0,0,2,2,1,0,0,1,0,2,0,1,0,0,0,0,0,0
3,4,0,1,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,1,0,0,2,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,5,4,1,0,0,0,0
4,1,0,0,0,4,0,2,3,0,0,0,0,0,0,0,0,0,0,0,6,5,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
2463,1,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2464,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2465,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0


# 1.2 Tfidf (term frequency * inverse document frequency)
+ Assign a weight to each term in each document rather than a raw count. 
+ gives more weight to less frequent terms
+ similar paremeters for this vectorizer 

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

In [0]:
t_vectorizer = TfidfVectorizer(ngram_range=(1, 3),  
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6,
                                   max_features=2000)


# call `fit` to build the vocabulary and calculate weights
t_vectorizer.fit(cleaned_talks)

# finally, call `transform` to convert text to a bag of words
tfidf_data = t_vectorizer.transform(cleaned_talks)

In [0]:
# view a dense representation of the document-term matrix
pd.DataFrame(tfidf_data.toarray(), columns=t_vectorizer.get_feature_names())

,ability,able,absolutely,abuse,accept,access,accident,according,account,achieve,act,action,active,activist,activity,actor,actual,add,added,addition,address,adult,advance,advanced,advantage,advice,affect,affected,afford,afghanistan,afraid,africa,african,age,agency,aging,ago,agree,agriculture,ah,...,wire,wisdom,wish,woman,wonder,wonderful,wondering,wood,word,worked,worker,working,world war,worldwide,worried,worry,worse,worst,worth,wow,write,writer,writing,written,wrong,wrote,year ago,year later,year old,year year,yellow,yes,yesterday,york,york city,young,young people,youtube,zero,zone
0,0.039900,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.027645,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.025223,0.000000,0.000000,0.000000,0.031826,0.0,0.0,0.027918,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.012701,0.000000,0.0,0.0,...,0.00000,0.000000,0.000000,0.083014,0.0,0.105703,0.0,0.000000,0.015117,0.000000,0.000000,0.000000,0.0,0.029783,0.0,0.0,0.000000,0.025423,0.022912,0.000000,0.021646,0.000000,0.0,0.000000,0.122141,0.021208,0.014169,0.000000,0.017664,0.000000,0.000000,0.000000,0.059567,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.071341,0.000000,0.000000,0.0,0.000000,0.029234,0.000000,0.000000,0.033446,0.000000,0.000000,0.000000,0.000000,0.040607,0.0,0.0,0.000000,0.000000,0.0,0.028812,0.000000,0.000000,0.000000,0.0,0.016206,0.000000,0.0,0.0,...,0.04129,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.018141,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.038866,0.0,0.031452,0.044525,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022344,0.038000,0.000000,0.000000,0.022075,0.038391,0.000000,0.0,0.0
2,0.000000,0.000000,0.056404,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.061131,0.026098,0.000000,0.046626,0.000000,0.000000,0.000000,0.000000,0.028305,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.015681,0.000000,0.0,0.011296,0.000000,0.0,0.0,...,0.00000,0.027805,0.000000,0.014766,0.0,0.018801,0.0,0.000000,0.161330,0.000000,0.000000,0.012645,0.0,0.000000,0.0,0.0,0.020903,0.000000,0.020377,0.000000,0.115508,0.000000,0.0,0.000000,0.031036,0.037724,0.012601,0.000000,0.000000,0.024352,0.000000,0.031150,0.000000,0.019109,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.021966,0.051184,0.018911,0.0,0.000000,0.018002,0.023057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.021935,0.000000,0.0,0.000000,0.000000,0.000000,0.071171,0.0,0.000000,0.000000,0.0,0.0,...,0.00000,0.000000,0.000000,0.013045,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.033514,0.0,0.000000,0.0,0.0,0.000000,0.019975,0.018002,0.000000,0.000000,0.000000,0.0,0.000000,0.013710,0.016664,0.000000,0.018503,0.000000,0.000000,0.000000,0.000000,0.000000,0.084408,0.096432,0.013594,0.000000,0.000000,0.0,0.0
4,0.000000,0.019273,0.000000,0.0,0.020959,0.031432,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.021358,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.023405,0.0,0.262969,0.078665,0.012833,0.021976,0.0,0.009244,0.000000,0.0,0.0,...,0.00000,0.000000,0.000000,0.024168,0.0,0.015387,0.0,0.000000,0.000000,0.000000,0.020465,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.015755,0.000000,0.0,0.000000,0.000000,0.000000,0.010313,0.000000,0.000000,0.000000,0.020927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

# 2 Topic modeling 
Use the document term matrix created with vectorization, to create a latent space and find the words that tend to ocurr together

We will use LDA Latent Dirichlet Allocation here (there are several methods, NMF, SVD)

This will reduce the data from thousands of terms (dimensions) to 20 topics. (Dimensionality Reduction)

Creates a latent space that is X dimensions.

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

# 2.1 LDA Latent Dirichlet Allocation

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [0]:
def topic_mod(vectorizer, vect_data, topics=20, iters=5, no_top_words=50):
    
    """ use Latent Dirichlet Allocation to get topics"""

    mod = LatentDirichletAllocation(n_components=topics,
                                    max_iter=iters,
                                    random_state=42,
                                    learning_method='online',
                                    n_jobs=-1)
    
    mod_dat = mod.fit_transform(vect_data)
    
    
    # to display a list of topic words and their scores 
    
    def display_topics(model, feature_names, no_top_words):
        for ix, topic in enumerate(model.components_):
            print("Topic ", ix)
            print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]) + '\n')
    
    display_topics(mod, vectorizer.get_feature_names() , no_top_words)

    
    return mod, mod_dat

In [0]:
lda_model, lda_data = topic_mod(c_vectorizer, 
                            count_vect_data, 
                            topics=20, 
                            iters=10, 
                            no_top_words=15)  

# 3 Visualization  

# 3.1 pyLDAvis 
+ plot the first 2 components from the topic modeling (LDA). 
+ Not really the best way to look at clusters, but a good place to start and a very nice way to present data to clients

In [0]:
# Setup to run in Jupyter notebook
pyLDAvis.enable_notebook()

 # Create the visualization
vis = pyLDAvis.sklearn.prepare(lda_model, count_vect_data, c_vectorizer, sort_topics=False, mds='mmds')

 # can export as a standalone HTML web page
pyLDAvis.save_html(vis, '/gdrive/My Drive/JBCS_NLP_intro/lda_visual.html')

# # Let's view it!
display(vis)

TypeError: ignored

# 3.3 Assign a topic to each document

+ for each document, assign the topic (column) with the  highest score from the LDA


In [0]:
topic_index = np.argmax(lda_data, axis=1)


 # Assign labels to topics
 Try to use the top terms from each topic to give it a label.

In [0]:
topic_names = pd.DataFrame(topic_index)

topic_names[topic_names==0] = "family"
topic_names[topic_names==1] = "agriculture"
topic_names[topic_names==2] = "space"
topic_names[topic_names==3] = "environment"
topic_names[topic_names==4] = "global economy"
topic_names[topic_names==5] = "writing"
topic_names[topic_names==6] = "sounds"
topic_names[topic_names==7] = "belief, mortality"
topic_names[topic_names==8] = "transportation"

topic_names[topic_names==9] = "gaming"
topic_names[topic_names==10] = "architecture"
topic_names[topic_names==11] = "education"

topic_names[topic_names==12] = "neuroscience"
topic_names[topic_names==13] = "climate, energy"

topic_names[topic_names==14] = "politics"
topic_names[topic_names==15] = "robotics"  
topic_names[topic_names==16] = "disease biology"
topic_names[topic_names==17] = "medicine"
topic_names[topic_names==18] = "technology, privacy"
topic_names[topic_names==19] = "war"

In [0]:
topic_names.head()

# Recommender
we will use the Ted Talk metadata to add some information to our recommender. 

# import original talks and metadata
merge them on the 'url' column

In [0]:
ted_trans = pd.read_csv('/gdrive/My Drive/JBCS_NLP_intro/transcripts.csv', encoding = "UTF-8")  
ted_main = pd.read_csv('/gdrive/My Drive/JBCS_NLP_intro/ted_main.csv', encoding = "UTF-8")  

NameError: ignored

In [0]:
ted_all = pd.merge(ted_trans, right=ted_main, on='url')
ted_all.url = ted_all.url.astype('str',copy=False)

ted_all.head(50)

In [0]:
def get_recommendations(target_doc, num_of_recs, topics, data, topic_model, vectorizer, topic_model_data):
    
    new_vec = topic_model.transform(
        vectorizer.transform([target_doc]))
    
    nn = NearestNeighbors(n_neighbors=num_of_recs, metric='cosine', algorithm='brute')
    nn.fit(topic_model_data)
    
    results = nn.kneighbors(new_vec)
    
    recommend_list = results[1][0]
    scores = results[0]
                       
    ss = np.array(scores).flat
    for i, resp in enumerate(recommend_list):
        print('\n--- ID ---\n', + resp)
        print('--- distance ---\n', + ss[i])  
        print('--- topic ---')
        print(topics.iloc[resp,0])
        print(data.iloc[resp,1])
        print('--- teds tags ---')
        print(data.iloc[resp,-3])
        
    return recommend_list, ss   

In [0]:
rec_list, scores = get_recommendations(cleaned_talks[804], num_of_recs=10, topics=topic_names, data=ted_all,
                                       topic_model=lda_model, vectorizer=c_vectorizer, topic_model_data=lda_data)

# search and recommend similar
We use a great library called fuzzywuzzy to find the titles that is most similar to a search term. then we use this as our target document for the recommendation. 

In [0]:
pip install fuzzywuzzy

In [0]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

search_term = "computer science"

titles = ted_all['title']

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


NameError: ignored

In [0]:
tite, score, talk_ind = process.extractOne(search_term, titles, scorer=fuzz.token_set_ratio)

In [0]:
rec_list, scores = get_recommendations(cleaned_talks[talk_ind], num_of_recs=10, topics=topic_names, data=ted_all,
                                       topic_model=lda_model, vectorizer=c_vectorizer, topic_model_data=lda_data)